In [1]:
import numpy as np
words_list=np.load('./wordsList.npy')
words_list=words_list.tolist()
words_list=[i.decode('UTF-8') for i in words_list]
words_vectors=np.load('./wordVectors.npy')


In [2]:
#可视化,40000*50维矩阵的第home_index维度数据
home_index=words_list.index('home')
home_vectors=words_vectors[home_index]
np.shape(words_vectors)

(400000, 50)

In [3]:
import matplotlib.pyplot
matplotlib.use('qt4agg')
matplotlib.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.pyplot.hist(num_words,50,facecolor='g')  #以x轴50为间隔划分
matplotlib.pyplot.xlabel='文本长度'
matplotlib.pyplot.ylabel='频次'
matplotlib.pyplot.xlim(0,1200)
matplotlib.pyplot.ylim(0,8000)
matplotlib.pyplot.show()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platf

NameError: name 'num_words' is not defined

In [4]:
import os
from os.path import isfile,join


#先合并文件路径名和文件名,判断是否为文件,若是则合并路径和名称加入列表
pos_files=['pos/'+f for f in os.listdir('pos/') if isfile(join('pos/',f))]
neg_files=['neg/'+f for f in os.listdir('neg/') if isfile(join('neg/',f))]
num_words=[]
for pf in pos_files:
    with open(pf,"r",encoding='utf-8') as f:
        line=f.readline()
        counter=len(line.split())
        num_words.append(counter)
print('正面评价完结')
for nf in neg_files:
    with open(nf, "r", encoding='utf-8') as f:
        line = f.readline()
        counter = len(line.split())
        num_words.append(counter)
print('负面评价完结')
num_files=len(num_words)
print('文件总数', num_files)
print('所有的词的数量', sum(num_words))
print('平均文件词的长度', sum(num_words) / len(num_words))

正面评价完结
负面评价完结
文件总数 25000
所有的词的数量 5844680
平均文件词的长度 233.7872


In [5]:
##标准化每个数据先把文本变为小写,再去除特殊字符

import re

strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
num_dimensions=50  # Dimensions for each word vector


def cleanSentences(string):
    string = string.lower().replace("<br />", " ")        #转化为小写
    return re.sub(strip_special_chars, "", string.lower()) #去除特殊字符
max_seq_num=250


In [89]:

# ids=np.zeros((num_files,max_seq_num),dtype='int32')
# file_count=0
# for pf in pos_files:
#     with open(pf,'r',encoding='utf-8') as f:
#         index=0
#         line=f.readline()
#         line=cleanSentences(line)
#         split=line.split()
#         for i in split:
#             try:
#                 ids[file_count][index]=words_list.index(i)
#             except ValueError:
#                 ids[file_count][index]=399999
#             index+=1
#             if index>=max_seq_num:
#                 break
#     file_count+=1
# for nf in neg_files:
#     with open(nf,'r',encoding='utf-8') as f:
#         index=0
#         line=f.readline()
#         line=cleanSentences(line)
#         split=line.split()
#         for i in split:
#             try:
#                 ids[file_count][index]=words_list.index(i)
#             except ValueError:
#                 ids[file_count][index]=399999
#             index+=1
#             if index>=max_seq_num:
#                 break
#     file_count+=1

# np.save('idsMatrix', ids)
                

In [6]:
ids=np.load('idsMatrix.npy')

In [7]:
from random import randint
def get_train_batch():
    labels=[]
    arr=np.zeros([batch_size,max_seq_num])
    for i in range(batch_size):
        if (i%2==0):
            num=randint(1,11499)
            labels.append([1,0])
        else:
            num=randint(13499,24999)
            labels.append([0,1])
        arr[i]=ids[num-1:num]
    return arr,labels
def get_test_batch():
    labels=[]
    arr=np.zeros([batch_size,max_seq_num])
    for i in range(batch_size):
        if (i%2==0):
            num=randint(11499,12499)
            labels.append([1,0])
        else:
            num=randint(12499,13499)
            labels.append([0,1])
        arr[i]=ids[num-1:num]
    return arr,labels


In [8]:
batch_size=32
lstm_units=64
num_labels=2
iterations=200000
lr=0.001

In [10]:
import tensorflow as tf

tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batch_size, num_labels])
input_data = tf.placeholder(tf.int32, [batch_size, max_seq_num])
data = tf.Variable(
    tf.zeros([batch_size, max_seq_num, num_dimensions]), dtype=tf.float32)
data = tf.nn.embedding_lookup(words_vectors, input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.5)
value, final = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstm_units, num_labels]))
bias = tf.Variable(tf.constant(0.1, shape=[num_labels]))
# value = tf.transpose(value, [1, 0, 2])
# last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(final[1], weight) + bias)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

saver = tf.train.Saver()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
import datetime
sess=tf.InteractiveSession()
tf.device('gpu/0')
saver=tf.train.Saver()
sess.run(tf.global_variables_initializer())
tf.summary.scalar('Loss',loss)
tf.summary.scalar('Accuracy',accuracy)
merged=tf.summary.merge_all()
logdir='tensorboard/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')+'/'
writer=tf.summary.FileWriter(logdir,sess.graph)

In [17]:
for i in range(iterations):
    nextbatch,nextlabel=get_train_batch()
    loss_,train_accuracy=sess.run([loss,accuracy], {input_data:nextbatch,labels:nextlabel})
    sess.run(optimizer,{input_data:nextbatch,labels:nextlabel})
    if (i%50==0):
        summary=sess.run(merged,{input_data:nextbatch,labels:nextlabel})
        writer.add_summary(summary,i)
        print(i,loss_,train_accuracy)
        
    if (i%10000==0 and i!=0):
        save_path=saver.save(sess,'models/pre_lstm.ckpt',global_step=i)
        print('Save to %s'%save_path)
writer.close()
    
    

0 0.8200028 0.5625
50 0.7413819 0.5
100 0.7163877 0.46875
150 0.71228147 0.46875
200 0.65460044 0.46875
250 0.6399676 0.625
300 0.6388531 0.59375
350 0.6715529 0.5625
400 0.5181569 0.6875
450 0.7576809 0.5
500 0.6747762 0.5625
550 0.71024936 0.5625
600 0.67685044 0.5625
650 0.6965557 0.53125
700 0.7254701 0.40625
750 0.68681747 0.5
800 0.6502071 0.5625
850 0.6815349 0.5
900 0.69612145 0.53125
950 0.6006578 0.65625
1000 0.66858184 0.625
1050 0.6978148 0.46875
1100 0.6835644 0.4375
1150 0.68731475 0.59375
1200 0.65789443 0.5625
1250 0.69507176 0.5
1300 0.69485736 0.53125
1350 0.75821984 0.53125
1400 0.63687277 0.65625
1450 0.6192664 0.625
1500 0.69420606 0.5625
1550 0.6623504 0.53125
1600 0.67524743 0.6875
1650 0.59349406 0.71875
1700 0.6818021 0.59375
1750 0.6628534 0.65625
1800 0.7488997 0.5625
1850 0.75030714 0.46875
1900 0.65105104 0.59375
1950 0.7341051 0.40625
2000 0.66759205 0.625
2050 0.7257222 0.46875
2100 0.69724596 0.5
2150 0.6371708 0.59375
2200 0.6683874 0.625
2250 0.6504108

17700 0.08152277 0.96875
17750 0.08242741 1.0
17800 0.09989707 0.96875
17850 0.25476205 0.90625
17900 0.2931702 0.90625
17950 0.1136674 0.96875
18000 0.09636469 0.96875
18050 0.1738947 0.9375
18100 0.090437084 0.96875
18150 0.03873167 1.0
18200 0.27907375 0.90625
18250 0.09350639 1.0
18300 0.44403508 0.90625
18350 0.2861445 0.875
18400 0.026732244 1.0
18450 0.10614034 0.96875
18500 0.04102849 1.0
18550 0.09772597 0.96875
18600 0.08578954 1.0
18650 0.2963412 0.90625
18700 0.049826436 1.0
18750 0.053497322 0.96875
18800 0.24285474 0.90625
18850 0.15521209 0.9375
18900 0.17964186 0.9375
18950 0.20607194 0.9375
19000 0.033375196 1.0
19050 0.06944688 0.96875
19100 0.039457735 1.0
19150 0.05469713 1.0
19200 0.085486606 0.9375
19250 0.08142469 0.96875
19300 0.06284824 1.0
19350 0.040905237 0.96875
19400 0.047056675 1.0
19450 0.015220555 1.0
19500 0.08099386 0.9375
19550 0.018816382 1.0
19600 0.04073213 1.0
19650 0.047805633 1.0
19700 0.13273686 0.96875
19750 0.05927027 0.96875
19800 0.1445058

35200 0.0014875224 1.0
35250 0.013332288 1.0
35300 0.010065846 1.0
35350 0.05349965 0.96875
35400 0.0025110547 1.0
35450 0.006368627 1.0
35500 0.0032278975 1.0
35550 0.0061800107 1.0
35600 0.02953846 0.96875
35650 0.01698541 1.0
35700 0.012480667 1.0
35750 0.008165053 1.0
35800 0.025881916 1.0
35850 0.057588864 0.96875
35900 0.22932799 0.96875
35950 0.053943116 0.96875
36000 0.03055915 1.0
36050 0.033377312 1.0
36100 0.03999903 0.96875
36150 0.0087773865 1.0
36200 0.0144095 1.0
36250 0.0091662025 1.0
36300 0.006012027 1.0
36350 0.017140383 1.0
36400 0.0036821393 1.0
36450 0.00068035204 1.0
36500 0.022707332 1.0
36550 0.28806812 0.96875
36600 0.009275924 1.0
36650 0.015955448 1.0
36700 0.0035858126 1.0
36750 0.006133966 1.0
36800 0.002840275 1.0
36850 0.033408344 0.96875
36900 0.07922717 0.96875
36950 0.033964206 0.96875
37000 0.046747364 0.96875
37050 0.00312551 1.0
37100 0.010550021 1.0
37150 0.003846004 1.0
37200 0.012862201 1.0
37250 0.0067211753 1.0
37300 0.004881079 1.0
37350 0.00

52900 0.003425316 1.0
52950 0.0014667113 1.0
53000 0.007829513 1.0
53050 0.011415801 1.0
53100 0.0026154688 1.0
53150 0.005374152 1.0
53200 0.0035689753 1.0
53250 0.0026353968 1.0
53300 0.009027494 1.0
53350 0.0025581303 1.0
53400 0.0016173247 1.0
53450 0.005982549 1.0
53500 0.0048706136 1.0
53550 0.00089681946 1.0
53600 0.016080035 1.0
53650 0.0017268902 1.0
53700 0.003035082 1.0
53750 0.0016077652 1.0
53800 0.004263403 1.0
53850 0.05736723 0.96875
53900 0.08138804 0.96875
53950 0.0027020765 1.0
54000 0.0013128256 1.0
54050 0.0015284801 1.0
54100 0.24477582 0.96875
54150 0.033496942 0.96875
54200 0.1006001 0.96875
54250 0.0021732738 1.0
54300 0.0013364137 1.0
54350 0.005892366 1.0
54400 0.0005318192 1.0
54450 0.00929038 1.0
54500 0.025728583 0.96875
54550 0.023720138 1.0
54600 0.0014919769 1.0
54650 0.0023924303 1.0
54700 0.0026779715 1.0
54750 0.0011825417 1.0
54800 0.007866556 1.0
54850 0.0010417572 1.0
54900 0.0031232242 1.0
54950 0.0016095568 1.0
55000 0.0046400493 1.0
55050 0.002

70650 0.06678971 0.96875
70700 0.010527937 1.0
70750 0.0045576077 1.0
70800 0.0015650775 1.0
70850 0.0015762516 1.0
70900 0.007802881 1.0
70950 0.0029268963 1.0
71000 0.0007252882 1.0
71050 0.003968856 1.0
71100 0.000731361 1.0
71150 0.0054290374 1.0
71200 0.0067506945 1.0
71250 0.0013983404 1.0
71300 0.005124391 1.0
71350 0.0021263766 1.0
71400 0.0014527128 1.0
71450 0.017071698 1.0
71500 0.04219146 0.96875
71550 0.0004728962 1.0
71600 0.0023402257 1.0
71650 0.0017525014 1.0
71700 0.0007593166 1.0
71750 0.0017846578 1.0
71800 0.0013378591 1.0
71850 0.19061378 0.96875
71900 0.1251089 0.9375
71950 0.00049459294 1.0
72000 0.0013663558 1.0
72050 0.040786717 0.96875
72100 0.0012204885 1.0
72150 0.0015946405 1.0
72200 0.0008381015 1.0
72250 0.00049758714 1.0
72300 0.0033544877 1.0
72350 0.0008139648 1.0
72400 0.00075563835 1.0
72450 0.00085376896 1.0
72500 0.027409328 1.0
72550 0.007842209 1.0
72600 0.035355244 0.96875
72650 0.00050864764 1.0
72700 0.07743502 0.96875
72750 0.00030216316 1.0

...

122450 0.019510714 1.0
122500 0.00054631993 1.0
122550 0.004829394 1.0
122600 0.0007019187 1.0
122650 0.00038608394 1.0
122700 0.002527909 1.0
122750 0.022374336 1.0
122800 0.0025335888 1.0
122850 0.0019955568 1.0
122900 0.042168356 0.96875
122950 0.0021935592 1.0
123000 0.39940974 0.9375
123050 0.002441092 1.0
123100 0.0013460112 1.0
123150 0.0012126114 1.0
123200 0.0012294231 1.0
123250 0.005403671 1.0
123300 0.00012610639 1.0
123350 0.0019103682 1.0
123400 0.00038334838 1.0
123450 0.0005070966 1.0
123500 0.00019585974 1.0
123550 0.0005178246 1.0
123600 0.0010711564 1.0
123650 0.0012140523 1.0
123700 0.0027127 1.0
123750 0.001424747 1.0
123800 0.0003622427 1.0
123850 0.0050688237 1.0
123900 0.0039817495 1.0
123950 0.0010826016 1.0
124000 0.0005179357 1.0
124050 0.0014705426 1.0
124100 0.00067662046 1.0
124150 0.00038645518 1.0
124200 0.0005868482 1.0
124250 0.0009602022 1.0
124300 0.00017102034 1.0
124350 0.0012898594 1.0
124400 0.077010624 0.96875
124450 0.00027128746 1.0
124500 0.0

139250 0.0033516763 1.0
139300 0.00023546889 1.0
139350 0.0018949115 1.0
139400 0.0012475199 1.0
139450 0.052079353 0.96875
139500 0.0017617507 1.0
139550 0.005817097 1.0
139600 0.0016292944 1.0
139650 0.00011534894 1.0
139700 0.0016907966 1.0
139750 0.00018117118 1.0
139800 0.00031477274 1.0
139850 0.00067105953 1.0
139900 0.00051374617 1.0
139950 9.496708e-05 1.0
140000 0.00030078704 1.0
Save to models/pre_lstm.ckpt-140000
140050 0.0011236822 1.0
140100 0.0033190162 1.0
140150 0.00030860474 1.0
140200 0.00041518224 1.0
140250 0.00059079914 1.0
140300 0.00067121664 1.0
140350 0.00071580836 1.0
140400 0.00032382202 1.0
140450 0.0003356486 1.0
140500 0.00023178417 1.0
140550 0.00068959914 1.0
140600 0.00071118755 1.0
140650 0.29422742 0.96875
140700 0.00044440533 1.0
140750 0.0006795195 1.0
140800 0.0019500357 1.0
140850 0.00096858793 1.0
140900 0.00032821402 1.0
140950 0.0009542831 1.0
141000 0.019010775 1.0
141050 0.0017550865 1.0
141100 0.0006591657 1.0
141150 0.0004774639 1.0
141200

155950 0.0058739274 1.0
156000 0.0011167257 1.0
156050 0.0046696486 1.0
156100 0.01890018 1.0
156150 0.010536207 1.0
156200 0.0016049494 1.0
156250 0.0051726135 1.0
156300 0.005014007 1.0
156350 0.00018508564 1.0
156400 0.0014333831 1.0
156450 0.00049791776 1.0
156500 0.00039649554 1.0
156550 0.0007241267 1.0
156600 0.0005318841 1.0
156650 0.00416369 1.0
156700 6.1929175e-05 1.0
156750 0.004976426 1.0
156800 2.7375323e-05 1.0
156850 0.004127659 1.0
156900 0.0010856759 1.0
156950 0.00029320366 1.0
157000 0.00020571137 1.0
157050 0.039879624 0.96875
157100 0.0001925056 1.0
157150 0.0028748272 1.0
157200 0.00090248795 1.0
157250 0.24623513 0.9375
157300 0.0041557206 1.0
157350 0.0014506299 1.0
157400 0.00043959913 1.0
157450 0.0013617959 1.0
157500 0.0007693767 1.0
157550 0.000321971 1.0
157600 0.00097280345 1.0
157650 0.00033485948 1.0
157700 0.00032619335 1.0
157750 0.013180708 1.0
157800 0.0025111837 1.0
157850 0.0042235497 1.0
157900 0.0014904707 1.0
157950 0.0017008908 1.0
158000 0.0

172650 0.25583917 0.875
172700 0.0011157962 1.0
172750 0.0048617967 1.0
172800 0.00026930223 1.0
172850 6.070947e-05 1.0
172900 0.002791728 1.0
172950 0.00038937543 1.0
173000 0.0011885207 1.0
173050 0.00061951176 1.0
173100 0.0006314868 1.0
173150 0.0036240884 1.0
173200 0.007366516 1.0
173250 0.00090225844 1.0
173300 0.00014003536 1.0
173350 8.163189e-05 1.0
173400 0.00017983599 1.0
173450 0.017325833 1.0
173500 0.0026822074 1.0
173550 0.0012064682 1.0
173600 7.92047e-05 1.0
173650 0.005250542 1.0
173700 0.0023409692 1.0
173750 0.00045569392 1.0
173800 0.002151487 1.0
173850 0.00046613146 1.0
173900 0.00071257586 1.0
173950 0.00028821107 1.0
174000 0.00029488525 1.0
174050 0.00010644717 1.0
174100 0.0005174088 1.0
174150 0.0004126231 1.0
174200 0.0059197173 1.0
174250 0.000261863 1.0
174300 0.00038096268 1.0
174350 0.00017003133 1.0
174400 0.0001613426 1.0
174450 0.014184664 1.0
174500 0.008552136 1.0
174550 0.0020380137 1.0
174600 0.0033565282 1.0
174650 0.0096471375 1.0
174700 0.00

189450 0.00040598382 1.0
189500 0.002256195 1.0
189550 0.00011219947 1.0
189600 0.0026445154 1.0
189650 0.033909947 0.96875
189700 0.00031419087 1.0
189750 6.689998e-05 1.0
189800 0.009090866 1.0
189850 0.0017802706 1.0
189900 0.0033755885 1.0
189950 0.0002456461 1.0
190000 0.032951403 0.96875
Save to models/pre_lstm.ckpt-190000
190050 0.00016326804 1.0
190100 0.00013254593 1.0
190150 0.000244239 1.0
190200 0.0049861707 1.0
190250 0.00086246274 1.0
190300 7.487544e-05 1.0
190350 0.00026853158 1.0
190400 0.0002291452 1.0
190450 0.0012363136 1.0
190500 7.2301686e-05 1.0
190550 9.285923e-05 1.0
190600 0.0004352318 1.0
190650 0.001747024 1.0
190700 0.00012672362 1.0
190750 0.0024625123 1.0
190800 0.000104395585 1.0
190850 0.004850375 1.0
190900 0.006231107 1.0
190950 0.0019697149 1.0
191000 0.00034742235 1.0
191050 0.0016257545 1.0
191100 0.00014214063 1.0
191150 0.00016784504 1.0
191200 0.00015849137 1.0
191250 0.00070006214 1.0
191300 0.00076357106 1.0
191350 0.00019697807 1.0
191400 0.0

In [12]:
if os.path.exists("models") and os.path.exists("models/checkpoint"):
    saver.restore(sess, tf.train.latest_checkpoint('models'))
test_num=1000
for i in range(test_num):
    test_batch,test_label=get_test_batch()
    loss_,train_accuracy=sess.run([loss,accuracy], {input_data:test_batch,labels:test_label})
        
    if (i%10==0 and i!=0):
        print(i,loss_,train_accuracy)



INFO:tensorflow:Restoring parameters from models/pre_lstm.ckpt-190000
10 1.2429423 0.84375
20 1.8356656 0.8125
30 0.61523086 0.90625
40 1.4716679 0.8125
50 1.8564813 0.8125
60 1.6079111 0.8125
70 1.4959685 0.84375
80 1.1235914 0.8125
90 1.7353405 0.84375
100 0.5085188 0.875
110 0.6581687 0.84375
120 1.8909998 0.84375
130 1.3058078 0.78125
140 1.9812206 0.8125
150 1.0999315 0.84375
160 1.4986407 0.78125
170 2.2331831 0.78125
180 1.8661273 0.75
190 1.6985719 0.78125
200 1.4018964 0.8125
210 1.6504254 0.84375
220 1.4042482 0.84375
230 1.7460644 0.78125
240 0.72444355 0.84375
250 0.67122823 0.90625
260 1.538337 0.78125
270 1.6031537 0.875
280 1.9158999 0.71875
290 0.30583644 0.9375
300 1.3958995 0.84375
310 1.2797498 0.84375
320 2.3868475 0.8125
330 0.6712154 0.90625
340 1.5727223 0.8125
350 1.5107734 0.8125
360 1.2107975 0.875
370 1.6876793 0.8125
380 1.076377 0.84375
390 1.8662195 0.78125
400 0.6134273 0.90625
410 1.7306716 0.8125
420 2.0679307 0.71875
430 1.0611898 0.84375
440 0.5970843

KeyboardInterrupt: 